# Multisample branch: Master Workflow for single-normal multi-tumor

## Availalbe multilane workflows ( Array[Array[File]] )  in ../src/wdl/
* trimming (Optional)
    * Inputs are multi-lane fasta
    * Alignment/Workflow/Runtrim_sequences.wdl
    * Outputs are multi-lane
* alignment
    * Input option (select_first) inputs are multi-lane fasta
    * Alignment/Workflow/Runalignment.wdl
    * Outputs are multi-lane fasta
* merge
    * inputs are multi-lane fasta
    * Alignment/Workflow/RunMergeAligned.wdl
    * Outputs: Bam, Bai

## Available (Bam-Bai) merged functions
* dedup (Optional)
    * Inputs are Bam, Bai
    * Alignment/Tasks/dedup.wdl
    * Outputs: - Bam, Bai
* deliver alignment
    * Input option (select_first) Bam, Bai
    * DeliveryOfAlignment/Tasks/deliver_alignment.wdl
    * Output - script delivers to:              DeliveryFolder_Alignment
* realignment
    * Inputs: DeliverAlignOutputBams, DeliverAlignOutputBais
    * HaplotyperVC/Tasks/realignment.wdl
    * Outputs: OutputBams, OutPutBais
* Bqsr (Optional)
    * Inputs OutputBams, OutPutBais
    * HaplotyperVC/Tasks/bqsr.wdl
    * Outputs RecalTable, RecalibratedBam, RecalibratedBai
* haplotyper
    * Inputs (select_first) and RecalTable
    * HaplotyperVC/Tasks/haplotyper.wdl
    * Outputs: OutputVcf, OutputVcfIdx
* Vqsr (Optional)
    * Inputs: OutputVcf, OutputVcfIdx
    * HaplotyperVC/Tasks/vqsr.wdl
    * Outputs: OutputVcf, OutputVcfIdx
* deliver HaplotyperVC
    * Inputs: (select_first)
    * DeliveryOfHaplotyperVC/Tasks/deliver_HaplotyperVC.wdl
    * Outputs: script delivery to DeliveryFolder_HaplotyperVC
****

# Multi-sample, single normal workflow strategy
## Reuse the workflow of multilane workflows through merge
* call once with normal
* scatter with multisamples
## Use merged workflow *SomaticVCStandaloneWorkflow.wdl*
* scatter each tumor with normal
* delivery task: what is the form of final output ?
****

```bash
###############################################################################################
##		SomaticMasterMultiWorkflow.wdl        ##
###############################################################################################


import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunTrimSequences_MultiSample.wdl" as TRIM_MULTI_SAMPLE_AND_LANE
import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunAlignment_MultiSample.wdl" as ALIGN_MULTI_SAMPLE_AND_LANE
import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunMergeAligned_MultiSample.wdl" as MERGE_MULTI_SAMPLE

workflow SomaticVCMultiSampleMasterWF {
        
   Array[Array[Array[File]]] InputReads 
	
   call TRIM_MULTI_SAMPLE_AND_LANE.RunMultiSample_Trimming as TrimMultiSampleMultiLane {
      input:
         InputReads = InputReads	
   }

   call ALIGN_MULTI_SAMPLE_AND_LANE.RunMultiSample_Alignment as AlignMultiSampleMultiLane {
      input:
         InputReads = TrimMultiSampleMultiLane.Outputs
   }

   call MERGE_MULTI_SAMPLE.RunMultiSampleMerge as MergeMultiSample {
      input:
         AlignedMultiSampleOutputBams = AlignMultiSampleMultiLane.OutputBams,
         AlignedMultiSampleOutputBais = AlignMultiSampleMultiLane.OutputBais
   }
}
```
****
## dedup & deliver -- then scatter pairs on *SomaticVCStandaloneWorkflow.wdl*
****

```bash
###############################################################################################
##              SomaticMultiSample.wdl                                                               ##
###############################################################################################
import "MayomicsVC/src/wdl/Alignment/Workflow/Runtrim_sequences.wdl" as CUTADAPTTRIM
import "MayomicsVC/src/wdl/Alignment/Workflow/Runalignment.wdl" as ALIGNMENT
import "MayomicsVC/src/wdl/Alignment/Tasks/merge_aligned_bam.wdl" as MERGEBAM
import "MayomicsVC/src/wdl/Alignment/Tasks/dedup.wdl" as DEDUP
import "MayomicsVC/src/wdl/DeliveryOfAlignment/Tasks/deliver_alignment.wdl" as DELIVER_Alignment

workflow SomaticVCWorkflow {

   Array[Array[File]] NormalInputReads
   String SampleNameNormal
   Array[Array[Array[File]]] TumorInputReads
   Array[String] PairSamplesName
   Array[Int] Index = range(length(TumorInputReads))

   Boolean PairedEnd
   Boolean Trimming
   Boolean Manta
   Boolean MarkDuplicates
   Boolean Bqsr
   Boolean Vqsr

   ######                               Normal sample      ######
   ##############################################################
   if(Trimming) {
      call CUTADAPTTRIM.RunTrimSequencesTask as NormalTrimseq {
         input:
            InputReads = NormalInputReads
      }
   }

   Array[Array[File]] NormalAlignInputReads = select_first( [NormalTrimseq.Outputs, NormalInputReads] )

   call ALIGNMENT.RunAlignmentTask as NormalAlign {
      input:
         SampleName = SampleNameNormal,
         InputReads = NormalAlignInputReads
   }

   call MERGEBAM.mergebamTask as NormalMergeBam {
      input:
         SampleName = SampleNameNormal,
         InputBams = NormalAlign.OutputBams,
         InputBais = NormalAlign.OutputBais
   }

   if(MarkDuplicates) {

       call DEDUP.dedupTask as NormalDedup {
          input:
             SampleName = SampleNameNormal,
             InputBams = NormalMergeBam.OutputBams,
             InputBais = NormalMergeBam.OutputBais
       }
   }

   File DeliverNormalAlignOutputBams = select_first( [NormalDedup.OutputBams, NormalMergeBam.OutputBams] )
   File DeliverNormalAlignOutputBais = select_first( [NormalDedup.OutputBais, NormalMergeBam.OutputBais] )


   call DELIVER_Alignment.deliverAlignmentTask as NormalDAB {
      input:
         SampleName = SampleNameNormal,
         InputBams = DeliverNormalAlignOutputBams,
         InputBais = DeliverNormalAlignOutputBais
   }
   
   ##############################################################
   ######                scatter block: Tumor samples      ######

   scatter (idx in Index) {

      String SampleName = PairSamplesName[idx]

      if(Trimming) {
          call CUTADAPTTRIM.RunTrimSequencesTask as TumorTrimseq {
             input:
                PairedEnd = PairedEnd,
                InputReads = TumorInputReads[idx],
                SampleName = SampleName
          }
      }

      Array[Array[File]] TumorAlignInputReads = select_first([TumorTrimseq.Outputs, TumorInputReads[idx]])

      call ALIGNMENT.RunAlignmentTask as TumorAlign {
         input:
            PairedEnd = PairedEnd,
            InputReads = TumorAlignInputReads,
            SampleName = SampleName
      }

      call MERGEBAM.mergebamTask as TumorMergeBam {
         input:
            SampleName = SampleName,
            InputBams = TumorAlign.OutputBams,
            InputBais = TumorAlign.OutputBais
      }

      if(MarkDuplicates) {

         call DEDUP.dedupTask as TumorDedup {
            input:
               SampleName = SampleName,
               InputBams = TumorMergeBam.OutputBams,
               InputBais = TumorMergeBam.OutputBais
          }
      }

      File DeliverTumorAlignOutputBams = select_first([TumorDedup.OutputBams, TumorMergeBam.OutputBams])
      File DeliverTumorAlignOutputBais = select_first([TumorDedup.OutputBais, TumorMergeBam.OutputBais])

      call DELIVER_Alignment.deliverAlignmentTask as TumorDAB {
         input:
            SampleName = SampleName,
            InputBams = DeliverNormalAlignOutputBams,
            InputBais = DeliverNormalAlignOutputBais
      }
   }
}
   
```

## New and Improved Version # 3
* how to switch on index + 1 etc...
